In [1]:
from google.colab import drive

drive.mount('/content/drive/')

ModuleNotFoundError: No module named 'google.colab'

In [187]:
import pandas as pd

In [188]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType, FloatType
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('ML_Project').setMaster('local').set("spark.executor.memory", "6g")
sc = pyspark.SparkContext(conf=conf)
#Create SparkSession
spark = SparkSession(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=ML_Project, master=local) created by __init__ at <ipython-input-2-189f8250b048>:10 

In [189]:
# import collision_road matching data with segment IDs
df_collision_segment_pairs = pd.read_csv('../data/processed/col_segment_pairs.csv')

# import collision data
df_collisions = pd.read_csv('../data/processed/collisions.csv')

# load all segment_ID in Berlin
df_segments = pd.read_csv('../data/processed/road_segments.csv')

c:\users\bened\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (5,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [190]:
# Retain only segment IDs from full segment list
df_segment_ids = df_segments[['segment_id']]
df_segment_ids

,segment_id
0,0
1,1
2,2
3,3
4,4
...,...
43105,43105
43106,43106
43107,43107
43108,43108


In [202]:
# Merge Collision Segment pairs with collision data set to get positive sample
df_positive_sample = df_collision_segment_pairs.merge(df_collisions, left_on='col_id', right_on='col_id', suffixes=('','_y'), copy=False)
df_positive_sample

,col_id,segment_id,collision_cnt,objectid,land,district,LOR,street,LOR_ab_2021,year,...,foot,motor,vehicle,other,road_con,LINREFX,LINREFY,XGCSWGS84,YGCSWGS84,geometry
0,1,41373.0,1,112705,11,12,12304314.0,Ziekowstraße/Freie Scholle,12500824.0,2018,...,0,0,0,0,0,"790639,5854","5835009,589",13.291022,52.587259,POINT (13.29102205 52.58725906)
1,3,828.0,6,112737,11,7,7020202.0,Volkspark (Rudolf-Wilde-Park),7200308.0,2018,...,1,0,0,0,0,"795223,509","5823525,402",13.348288,52.481844,POINT (13.34828776 52.48184447)
2,4,21397.0,2,112747,11,3,3020209.0,Niederschönhausen,3200206.0,2018,...,1,0,0,0,1,"798261,3849","5835047,26",13.403228,52.583472,POINT (13.40322797 52.58347154)
3,6,41696.0,1,112792,11,4,4061757.0,Forst Grunewald,4200207.0,2018,...,0,0,0,1,1,"788961,8201","5823583,542",13.256390,52.485715,POINT (13.25639043 52.48571541)
4,7,42255.0,2,112805,11,3,3010104.0,Lietzengraben,3100101.0,2018,...,0,0,0,0,1,"799543,5822","5840743,375",13.427252,52.633823,POINT (13.42725182 52.63382261)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52113,38472,21737.0,6,145557,11,1,1044201.0,NaN,1401045.0,2020,...,0,0,0,0,0,"796697,6855","5831916,881",13.377416,52.556266,POINT (13.37741627 52.55626636)
52114,38511,2782.0,20,145596,11,1,1044201.0,NaN,1401045.0,2020,...,0,0,0,0,1,"797094,9703","5831923,772",13.383265,52.556112,POINT (13.38326526 52.55611156)
52115,38536,2781.0,21,145621,11,1,1044201.0,NaN,1401045.0,2020,...,0,0,0,1,0,"796599,2162","5831923,893",13.375974,52.556383,POINT (13.37597435 52.55638286)
52116,38556,19720.0,14,145641,11,3,3061228.0,NaN,3601245.0,2020,...,0,1,0,0,0,"800403,9773","5831533,36",13.431574,52.550797,POINT (13.43157439 52.5507971)


In [192]:
# Some type conversions
df_segments.bezirk = df_segments.bezirk.astype(str)
df_segments.stadtteil = df_segments.stadtteil.astype(str)
df_segments.strassenna = df_segments.strassenna.astype(str)
df_segments.str_bez = df_segments.str_bez.astype(str)

df_positive_sample.street = df_positive_sample.street.astype(str)
df_positive_sample.other = df_positive_sample.other.astype(str)
df_positive_sample.road_con = df_positive_sample.road_con.astype(str)



In [193]:
# generate all dates from start to end
import time
import datetime



start = "31/12/2017"
end = "31/12/2020"
date = datetime.datetime.strptime(start, "%d/%m/%Y")

dates = list()
while(date != datetime.datetime.strptime(end, "%d/%m/%Y")): 
    date += datetime.timedelta(days=1)
    for i in range(24):
        dates.append((date.strftime("%Y-%m-%d"), i))

In [194]:
# Ge date hour pairs and preprocess them to fit with formatting of collision data (e.g. Sunday: 1)
df_date_hour_pairs = pd.DataFrame(dates, columns = ['date', 'hour'])
df_date_hour_pairs['date'] = pd.to_datetime(df_date_hour_pairs['date'])
df_date_hour_pairs['year'] = df_date_hour_pairs['date'].dt.year
df_date_hour_pairs['month'] = df_date_hour_pairs['date'].dt.month
df_date_hour_pairs['weekday'] = df_date_hour_pairs['date'].dt.weekday + 2
df_date_hour_pairs.loc[(df_date_hour_pairs.weekday == 8),'weekday']=1
df_date_hour_pairs = df_date_hour_pairs.drop('date', axis=1)
df_date_hour_pairs = df_date_hour_pairs.drop_duplicates()
df_date_hour_pairs

,hour,year,month,weekday
0,0,2018,1,2
1,1,2018,1,2
2,2,2018,1,2
3,3,2018,1,2
4,4,2018,1,2
...,...,...,...,...
25723,19,2020,12,2
25724,20,2020,12,2
25725,21,2020,12,2
25726,22,2020,12,2


In [195]:
df_date_hour_pairs = spark.createDataFrame(df_date_hour_pairs) 
df_segment_ids = spark.createDataFrame(df_segment_ids) 
df_positive_sample = spark.createDataFrame(df_positive_sample) 

In [196]:
# Create all possible year, month, day, hour, segment combinations for 2018-2020
df_combinations = df_date_hour_pairs.crossJoin(df_segment_ids).repartition(100)
df_combinations.count()

260729280

In [197]:
# Drop all accident time paris that acutally have been recorded as positive examples (collisions occurred)
df_neg_combinations = df_combinations.join(df_positive_sample,(df_combinations.year == df_positive_sample.year) & 
                                                            (df_combinations.month == df_positive_sample.month) & 
                                                            (df_combinations.weekday == df_positive_sample.weekday) & 
                                                            (df_combinations.hour == df_positive_sample.hour) &
                                                            (df_combinations.segment_id == df_positive_sample.segment_id), 
                                       "leftanti").repartition(100)
df_neg_combinations.count()

260677246

In [198]:
# Draw 0.1% uniform sample of negative combinations to replicate Hebert approach
df_neg_combinations = df_neg_combinations.sample(False, 0.001, seed=None)
df_neg_combinations.count()

260702

In [199]:
# Convert it to Pandas DF
df_neg_combinations = df_neg_combinations.toPandas()
df_positive_sample = df_positive_sample.toPandas()

In [203]:
df_negative_sample = df_neg_combinations.merge(df_positive_sample.drop_duplicates(['segment_id']), how="left", left_on=["segment_id"], right_on=["segment_id"],suffixes=('','_del')).drop(['year_del','month_del','hour_del','weekday_del'],axis=1)

In [204]:
# Generate new outcome column: "0" means non collision has occurred
df_negative_sample['collision'] = 0
df_positive_sample['collision'] = 1

In [205]:
# Add midpoint (geographic centers) of segments
df_negative_sample = df_negative_sample.merge(df_segments.drop_duplicates(['segment_id'])[['segment_id','mid_lat', 'mid_lon', 'length_m', 'side_strt']], how="left", left_on="segment_id", right_on="segment_id")
df_positive_sample = df_positive_sample.merge(df_segments.drop_duplicates(['segment_id'])[['segment_id','mid_lat', 'mid_lon', 'length_m', 'side_strt']], how="left", left_on=["segment_id"], right_on=["segment_id"])

In [209]:
df_negative_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260702 entries, 0 to 260701
Data columns (total 34 columns):
hour             260702 non-null int64
year             260702 non-null int64
month            260702 non-null int64
weekday          260702 non-null int64
segment_id       260702 non-null int64
col_id           90192 non-null float64
collision_cnt    90192 non-null float64
objectid         90192 non-null float64
land             90192 non-null float64
district         90192 non-null float64
LOR              90166 non-null float64
street           75476 non-null object
LOR_ab_2021      66053 non-null float64
acc_cat          90192 non-null float64
acc_type1        90192 non-null float64
acc_type2        90192 non-null float64
lightratio       90192 non-null float64
bike             90192 non-null float64
car              90192 non-null float64
foot             90192 non-null float64
motor            90192 non-null float64
vehicle          90192 non-null float64
other          

In [206]:
df_positive_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52118 entries, 0 to 52117
Data columns (total 34 columns):
col_id           52118 non-null int64
segment_id       52118 non-null float64
collision_cnt    52118 non-null int64
objectid         52118 non-null int64
land             52118 non-null int64
district         52118 non-null int64
LOR              52111 non-null float64
street           36227 non-null object
LOR_ab_2021      34216 non-null float64
year             52118 non-null int64
month            52118 non-null int64
hour             52118 non-null int64
weekday          52118 non-null int64
acc_cat          52118 non-null int64
acc_type1        52118 non-null int64
acc_type2        52118 non-null int64
lightratio       52118 non-null int64
bike             52118 non-null int64
car              52118 non-null int64
foot             52118 non-null int64
motor            52118 non-null int64
vehicle          52118 non-null int64
other            52118 non-null object
road_con 

In [211]:
# EXPORT
df_negative_sample.to_csv('../data/processed/negative_sample.csv')
df_positive_sample.to_csv('../data/processed/positive_sample.csv')